### Lite Loans Scorecard Development Plan Part 2 ####
Behavioral Analysis

  * Understanding Maximum delinquency event trends for Lite loans:
    * Default rates analysis 
    * Cure rates analysis
  * Define Target definition
    * Good/Bad flag

  
  


  
  
  
  

In [1]:
# Load Dependences

from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import datetime
import warnings
import re
from calendar import month_abbr
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

plt.style.use('ggplot')
%matplotlib inline
color = sns.color_palette()



warnings.filterwarnings("ignore")

# set display options
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

# Set plot styles
plt.style.use("ggplot")
%matplotlib inline
sns.set(style="whitegrid", color_codes=True)
color = sns.color_palette()

# Plotting pretty figures and avoid blurry images
%config InlineBackend.figure_format = 'retina'
# Larger scale for plots in notebooks
sns.set_context('talk')

%run utils.ipynb
print('libraries imported successfully')


Util Libraries successfully imported
libraries imported successfully


In [2]:
in_file_lite_loans = '\\\\10.0.10.20\\NetworkShare\\1. Active Projects\\Lite loans sc\\data\\processed\\1.0 lite loans data after audit.csv'

In [3]:
# read data
lite_loan_apps = pd.read_csv(in_file_lite_loans, encoding='unicode_escape')

# Ensuring we exclude any duplicates found in the entire population
lite_loan_apps = lite_loan_apps.drop_duplicates(subset='LoanId', keep='first')


In [4]:
lite_loan_apps.shape

(6544, 62)

In [4]:
#confirm we have unique customers in the data
lite_loan_apps.clientID.value_counts()

22509047810    1
22163436203    1
22344030910    1
22144297656    1
22143480507    1
22307842996    1
22496971444    1
22324392627    1
22327367778    1
22230977196    1
22153515695    1
22179635881    1
22370835110    1
22351186620    1
22178849441    1
22209225271    1
22217147037    1
22386249062    1
22321935003    1
22522561173    1
22276770452    1
22175830679    1
22338708113    1
22246484624    1
22274593471    1
22178242701    1
22274552355    1
22242065122    1
22252892923    1
22176112839    1
22145225463    1
22363005681    1
22306026227    1
22158631657    1
22251188971    1
22361064165    1
22206064479    1
22287528675    1
22200594215    1
22185446084    1
22198066127    1
22263452383    1
22373083864    1
22346674907    1
22199976661    1
22206108375    1
22257279697    1
22353134285    1
22165885647    1
22411385550    1
22150251149    1
22244954767    1
22215367310    1
22409218628    1
22151516759    1
22190778916    1
22153704019    1
22241264205    1
22413396556   

In [6]:
lite_loan_apps[lite_loan_apps['clientID']==22174373885]

,Unnamed: 0,BureauData,LoanId,AccountState,AccountSubstate,Product,CreationDate,DisbursementDate,DisbMonth,InitialDecision,InitialDecisionCode,ReasonCode_01,Age,DocumentType,DTI,EducationStatus,EmployerLGA,EmployerState,Gender,Income,Industry,LoanAmount,MaritalStatus,NoChildren,RepaymentBank,RepaymentMethod,AddresLGA,ResidentialStatus,Term,TimeEmploymentMM,TotalNumberAccountsInGoodStanding,TotalNumberOfAccounts,TotalMonthlyInstallments,TotalNumberOfAccountsInArrs,TotalNumberOfAccountsMoreThan90,HighestReportedMonthsInArrs,TotalNumberOfJudgments,TotalOutstandingDebt,TotalTimeAtAddress,WorstStatus,BusinessSector,EmploymentPosition,GBFlag,MaxDPD,(DueDate) FPD,FPD,Closeddate,MaturityDate,EarlySettlementFlg,WeekDay,DayOfTheMonth,timeindayselect,MobileNetwork,Email,ClientFullName,FirstRD,CreditScore,FirstDD,Channels,InterestRate,Acct_Duratn,clientID
948,58257,0,92020850,CLOSED,NaN,Lite Loan - New,2021-03-31,2021-04-03,2021-04-01,2021-03-31,Accept,NaN,34,NIN,0.000,Graduate,Eti-Osa,Lagos,FEMALE,300000.000,Services,20000.000,Married,2,UBA,Debit Card (+Cheque),Ajeromi-Ifelodun,Renting,1,94,-999,-999,-999.000,-999,-999,0,-999,-999.000,246.000,-999,Services,NaN,Good,-6.000,2021-05-03,PAID IN 7,2021-04-27,2021-06-02,1,Wednesday,31,4pm - 6pm,GLO,nkwochaaugusta@yahoo.com,Augusta Nkwocha,NaN,NaN,2021-05-03,Renmobile,30.000,NaN,22174373885


### Disbursement Analysis Breakdown
Not all through-the-door applications are considered for disbursements by virtue of the through-the-cycle process of onboarding Lite-loan customer applications. Accessing Credit typically goes through thorough verification process and policy rules set before an eventual decision is made on whether to either **Accept or Reject**. For Lite-loans currently, there is no credit scoring model to make automated business decisioning for this portfolio of loan accounts. 

In [7]:

# Count and dimension disbursements frequency by Disbursement Month
disbursed_pop = lite_loan_apps.groupby('DisbMonth', as_index=False)[
    'DisbursementDate'].count()
# Renaming from DisbursementDate to DisbFreq the column to avoid confusion
disbursed_pop = disbursed_pop.rename(columns={'DisbursementDate': 'DisbFreq'})


In [8]:
# convert DisbMonth into a datetime from a string object
disbursed_pop['DisbMonth'] = pd.to_datetime(disbursed_pop['DisbMonth'])
disbursed_pop.sort_values(by=['DisbMonth'], inplace=True, ignore_index=True)
# convert from datetime object to date string
disbursed_pop['DisbMonth'] = disbursed_pop['DisbMonth'].dt.strftime('%b-%Y')


In [9]:
disbursed_pop

,DisbMonth,DisbFreq
0,Feb-2021,203
1,Mar-2021,690
2,Apr-2021,665
3,May-2021,681
4,Jun-2021,639
5,Jul-2021,447
6,Aug-2021,768
7,Sep-2021,265
8,Oct-2021,712
9,Nov-2021,718


In [10]:
lite_loan_apps.AccountState.value_counts()

CLOSED                5305
ACTIVE_IN_ARREARS      681
CLOSED_WRITTEN_OFF     538
ACTIVE                  20
Name: AccountState, dtype: int64

#### Early Settlement Analysis
  We found out some customers get to pay only principal. We found out some of these customers pay back around two days after being disbursed to and some are the QA guys testing the product when it was introduced earlier in the year

  Our definition of Early settlement customers are customers that close their accounts 2 days after being disbursed to

In [11]:
# Lets try to convert date column from a string object into a datetime object
lite_loan_apps['(DueDate) FPD'] = pd.to_datetime(
    lite_loan_apps['(DueDate) FPD'])
lite_loan_apps['Closeddate'] = pd.to_datetime(lite_loan_apps['Closeddate'])
lite_loan_apps['MaturityDate'] = pd.to_datetime(lite_loan_apps['MaturityDate'])
lite_loan_apps['DisbursementDate'] = pd.to_datetime(
    lite_loan_apps['DisbursementDate'])


In [12]:
# get the difference between closeddate and disbursementdate
lite_loan_apps['Loan duration'] = (
    lite_loan_apps['Closeddate'] - lite_loan_apps['DisbursementDate']).dt.days

# flag early settled customers for those who paid less than 3 daysafter beig disbursed to

lite_loan_apps['Early Settled'] = np.where(
    lite_loan_apps['Loan duration'] < 3, 1, 0)


In [13]:
# Lets create a cross tabulation that outputs actual early settlements 1 and non-early settlements 1 per vintage
EarlySettlement = pd.crosstab(
    index=lite_loan_apps['DisbMonth'], columns=lite_loan_apps['Early Settled'], dropna=False)
EarlySettlement


Early Settled,0,1
DisbMonth,,
2021-02-01,200,3
2021-03-01,674,16
2021-04-01,650,15
2021-05-01,658,23
2021-06-01,621,18
2021-07-01,435,12
2021-08-01,750,18
2021-09-01,259,6
2021-10-01,703,9


In [14]:
lite_loan_apps[lite_loan_apps['Early Settled'] == 1].sample(5)


,Unnamed: 0,BureauData,LoanId,AccountState,AccountSubstate,Product,CreationDate,DisbursementDate,DisbMonth,InitialDecision,InitialDecisionCode,ReasonCode_01,Age,DocumentType,DTI,EducationStatus,EmployerLGA,EmployerState,Gender,Income,Industry,LoanAmount,MaritalStatus,NoChildren,RepaymentBank,RepaymentMethod,AddresLGA,ResidentialStatus,Term,TimeEmploymentMM,TotalNumberAccountsInGoodStanding,TotalNumberOfAccounts,TotalMonthlyInstallments,TotalNumberOfAccountsInArrs,TotalNumberOfAccountsMoreThan90,HighestReportedMonthsInArrs,TotalNumberOfJudgments,TotalOutstandingDebt,TotalTimeAtAddress,WorstStatus,BusinessSector,EmploymentPosition,GBFlag,MaxDPD,(DueDate) FPD,FPD,Closeddate,MaturityDate,EarlySettlementFlg,WeekDay,DayOfTheMonth,timeindayselect,MobileNetwork,Email,ClientFullName,FirstRD,CreditScore,FirstDD,Channels,InterestRate,Acct_Duratn,clientID,Loan duration,Early Settled
3703,169428,0,92107940,CLOSED,NaN,Lite Loan - New,2021-08-17,2021-08-18,2021-08-01,2021-08-17,Accept,NaN,30,NIN,0.000,Graduate,Lagos Island,Lagos,MALE,100000.000,Education,5000.000,Married,1,GT Bank,Debit Card (+Cheque),Ikorodu,Owner,1,51,-999,-999,-999.000,-999,-999,-999,-999,-999.000,30.000,-999,Education,NaN,Good,-28.000,2021-09-17,PAID IN 7,2021-08-20,2021-10-17,1,Tuesday,17,6am - 8am,MTN,michaelshotade@gmail.com,Michael Shotade,NaN,NaN,2021-09-17,Renmobile,30.000,NaN,22149363956,2.000,1
5144,92283,0,92159457,CLOSED,NaN,Lite Loan - New,2021-10-22,2021-11-03,2021-11-01,2021-10-22,Accept,NaN,34,BVN,0.000,HND,Oshodi-Isolo,Lagos,MALE,40000.000,Manufacturing,5000.000,Single,0,UBA,Debit Card (+Cheque),Surulere,Renting,1,46,-999,-999,-999.000,-999,-999,-999,-999,-999.000,370.000,-999,Manufacturing,NaN,Good,-29.000,2021-12-03,PAID IN 7,2021-11-04,2022-01-02,1,Friday,22,10am - 12pm,MTN,sugarboi2709@gmail.com,Amanam A Ebenezer,NaN,NaN,2021-12-03,Renmobile,30.000,NaN,22142690008,1.000,1
2979,144395,0,92073967,CLOSED,NaN,Lite Loan - New,2021-07-07,2021-07-07,2021-07-01,2021-07-07,Accept,NaN,33,Drivers License,0.000,Secondary,Agege,Lagos,MALE,55000.000,Manufacturing,6000.000,Married,1,GT Bank,Debit Card (+Cheque),Agege,Renting,1,31,-999,-999,-999.000,-999,-999,-999,-999,-999.000,178.000,-999,Manufacturing,NaN,Good,-29.000,2021-08-06,PAID IN 7,2021-07-08,2021-09-05,1,Wednesday,7,4pm - 6pm,AIRTEL,innocentokoro42@gmail.com,Friday Okoro,NaN,NaN,2021-08-06,Renmobile,30.000,NaN,22229962534,1.000,1
3119,97291,0,92078989,CLOSED,NaN,Lite Loan - New,2021-07-14,2021-07-14,2021-07-01,2021-07-14,Accept,NaN,36,Drivers License,0.000,Graduate,Port Harcourt,Rivers,MALE,400000.000,Construction,7800.000,Married,1,UBA,Debit Card (+Cheque),Port Harcourt,Renting,1,26,-999,-999,-999.000,-999,-999,-999,-999,-999.000,51.000,-999,Construction,NaN,Good,-29.000,2021-08-13,PAID IN 7,2021-07-15,2021-09-12,1,Wednesday,14,10am - 12pm,MTN,abrahamarthur27@yahoo.com,Abraham Chigozie Hayford,NaN,NaN,2021-08-13,Web App,30.000,NaN,22179666946,1.000,1
2664,71273,0,92038345,CLOSED,NaN,Lite Loan - New,2021-05-15,2021-06-22,2021-06-01,2021-05-15,Accept,NaN,29,International Passport,0.000,Graduate,Lagos Island,Lagos,FEMALE,121000.000,Wholesale and retail trade,12800.000,Single,0,GT Bank,Debit Card (+Cheque),Lagos Mainland,Renting,1,23,-999,-999,-999.000,-999,-999,-999,-999,-999.000,24.000,-999,Wholesale and retail trade,NaN,Good,-26.000,2021-07-19,PAID IN 7,2021-06-23,2021-08-17,1,Saturday,15,10am - 12pm,MTN,onyinyeudeh1992@gmail.com,Onyinye Maryrose Udeh,NaN,NaN,2021-07-19,Renmobile,30.000,NaN,22168611094,1.000,1


In [15]:
# Allocate as an index the DisbMonth column and format into a datetime object
EarlySettlement.index = pd.to_datetime(EarlySettlement.index)
# Sort attributes to follow conventional Month-on-month arrangement
EarlySettlement.sort_values(by=['DisbMonth'], inplace=True)
# Re-format the attribute values into a monthname-year format
EarlySettlement.index = EarlySettlement.index.strftime('%b-%Y')
# We obtain the rates of actual early settlement flg from the total disbursed population
EarlySettlement['ES_Rate'] = EarlySettlement[1]/EarlySettlement.sum(axis=1)

EarlySettlement.to_csv('early settlement-Lite.csv')


In [16]:
# filter our these early settled customers from our data

lite_loan_apps = lite_loan_apps[lite_loan_apps['Early Settled'] == 0]
lite_loan_apps.shape


(6395, 64)

#### Delinquency Performance Analysis 
   Delinquency Buckets performance gives us the true status of disbursed accounts for the period in consideration. This usually gives a picture of the maximum number of days that a particular customer on an account level has been in **'default'** or **'refused to pay'**.
  

In [17]:
lite_loan_apps.columns


Index(['Unnamed: 0', 'BureauData', 'LoanId', 'AccountState', 'AccountSubstate',
       'Product', 'CreationDate', 'DisbursementDate', 'DisbMonth',
       'InitialDecision', 'InitialDecisionCode', 'ReasonCode_01', 'Age',
       'DocumentType', 'DTI', 'EducationStatus', 'EmployerLGA',
       'EmployerState', 'Gender', 'Income', 'Industry', 'LoanAmount',
       'MaritalStatus', 'NoChildren', 'RepaymentBank', 'RepaymentMethod',
       'AddresLGA', 'ResidentialStatus', 'Term', 'TimeEmploymentMM',
       'TotalNumberAccountsInGoodStanding', 'TotalNumberOfAccounts',
       'TotalMonthlyInstallments', 'TotalNumberOfAccountsInArrs',
       'TotalNumberOfAccountsMoreThan90', 'HighestReportedMonthsInArrs',
       'TotalNumberOfJudgments', 'TotalOutstandingDebt', 'TotalTimeAtAddress',
       'WorstStatus', 'BusinessSector', 'EmploymentPosition', 'GBFlag',
       'MaxDPD', '(DueDate) FPD', 'FPD', 'Closeddate', 'MaturityDate',
       'EarlySettlementFlg', 'WeekDay', 'DayOfTheMonth', 'timeindayselect

#### Business Overview on Delinquency Events or  Credit Defaults behaviour 
In general for our default analysis, we have to bear in mind that not all disbursed population are considered. by segmentation we have identified accounts that settled off early i.e. before their **first due date** this accounts are not of interest to our business because of the lack of expected interest income or revenue for the duration of the loan cycle. Hence, we consider disbursed population only that made payments over their amortization schedule/period except that they did this before their maturity dates. 

#### Latest Modification based on Lite Loans Product Design
Based on interactions with key stakeholders such as Products and Risk Management Team and internal review within DS. There is a slight adjustment we need to adopt for the development of Credit Scoring model for Lite Loans. We discovered that indeed regardless of the repayment behavior of a customer, an interest of 30% is being paid upfront with customers paying their full principal immediately after. In reality lite loans is a one-month loan product but a 3-month loan tenor is communicated in the loan offer letter to the customer. A customer's default only starts to be accounted for if a customer misses out on repayment of both interest and principal after the first installment due date of the facility. In general, for the Lite loan product we don't require a segmentation of the settlement flag into **early settlement 1** or **early settlement 2**. Because we are able to collect our interest regardless of how a customer chooses to repay within their loan amortization scheduled.

In [18]:
lite_loan_apps.MaxDPD.max()


343.0

In [19]:
#del_portf = lite_loan_apps[(lite_loan_apps['MaxDPD'].notna()) & (lite_loan_apps['Early_Settlement_1'] != 1)]
del_portf = lite_loan_apps[(lite_loan_apps['MaxDPD'].notna())]


In [20]:
del_portf.shape


(6394, 64)

In [21]:
# del_portf['MaxDPD'].unique()


In [22]:
del_portf['MaxDPDBucket'] = np.where(del_portf['MaxDPD'] <= 10, '0-10', np.where((del_portf['MaxDPD'] >= 11)
                                                                                 & (del_portf['MaxDPD'] < 31), '11-30', np.where((del_portf['MaxDPD'] >= 31)
                                                                                                                                 & (del_portf['MaxDPD'] < 61), '31-60', np.where((del_portf['MaxDPD'] >= 61)
                                                                                                                                                                                 & (del_portf['MaxDPD'] < 91), '61-90', np.where((del_portf['MaxDPD'] >= 91)
                                                                                                                                                                                                                                 & (del_portf['MaxDPD'] < 121), '91-120', np.where((del_portf['MaxDPD'] > 120), '121-Above', del_portf['MaxDPD']))))))


In [23]:
del_portf.head()


,Unnamed: 0,BureauData,LoanId,AccountState,AccountSubstate,Product,CreationDate,DisbursementDate,DisbMonth,InitialDecision,InitialDecisionCode,ReasonCode_01,Age,DocumentType,DTI,EducationStatus,EmployerLGA,EmployerState,Gender,Income,Industry,LoanAmount,MaritalStatus,NoChildren,RepaymentBank,RepaymentMethod,AddresLGA,ResidentialStatus,Term,TimeEmploymentMM,TotalNumberAccountsInGoodStanding,TotalNumberOfAccounts,TotalMonthlyInstallments,TotalNumberOfAccountsInArrs,TotalNumberOfAccountsMoreThan90,HighestReportedMonthsInArrs,TotalNumberOfJudgments,TotalOutstandingDebt,TotalTimeAtAddress,WorstStatus,BusinessSector,EmploymentPosition,GBFlag,MaxDPD,(DueDate) FPD,FPD,Closeddate,MaturityDate,EarlySettlementFlg,WeekDay,DayOfTheMonth,timeindayselect,MobileNetwork,Email,ClientFullName,FirstRD,CreditScore,FirstDD,Channels,InterestRate,Acct_Duratn,clientID,Loan duration,Early Settled,MaxDPDBucket
0,219111,0,92002528,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,26,NIN,0.000,Graduate,Ikeja,Lagos,MALE,60000.000,Services,11300.000,Single,0,Zenith Bank,Debit Card (+Cheque),Alimosho,Renting,1,56,-999,-999,-999.000,-999,-999,-999,-999,-999.000,275.000,-999,Services,NaN,Good,0.000,2021-03-12,PAID IN 7,2021-03-12,2021-04-12,1,Friday,12,10am - 12pm,AIRTEL,drezyokoro@gmail.com,David Okoro,NaN,NaN,2021-03-12,Renmobile,30.000,NaN,22163766724,28.000,0,0-10
1,103666,0,92002589,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,28,Social Security Card,0.000,Graduate,Ikeja,Lagos,MALE,242000.000,Transportation and storage,20000.000,Single,0,Zenith Bank,Debit Card (+Cheque),Alimosho,Owner,1,54,-999,-999,-999.000,-999,-999,-999,-999,-999.000,313.000,-999,Transportation and storage,NaN,Bad,323.000,2021-03-12,PAID IN 7,2021-03-03,2021-05-12,1,Friday,12,2pm - 4pm,MTN,Chukwuedomichael@gmail.com,Michael Chukwuedo,NaN,NaN,2021-03-12,Web App,30.000,NaN,22150679002,19.000,0,121-Above
2,103656,0,92002560,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,42,Voters Identity Card,0.000,Graduate,Ikorodu,Lagos,MALE,85000.000,Services,10000.000,Married,1,GT Bank,Debit Card (+Cheque),Oredo,Renting,1,180,-999,-999,-999.000,-999,-999,-999,-999,-999.000,27.000,-999,Services,NaN,Good,6.000,2021-03-12,PAID IN 7,2021-03-18,2021-04-12,1,Friday,12,12pm - 2pm,MTN,popoolababatunde@rocketmail.com,Popoola Babatunde,NaN,NaN,2021-03-12,Web App,30.000,NaN,22161568021,34.000,0,0-10
3,103636,0,92002533,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,41,Drivers Licence,0.000,Post Graduate,Zaria,Kaduna,MALE,157567.000,Education,20000.000,Married,4,GT Bank,Debit Card (+Cheque),Kaduna,Owner,1,119,-999,-999,-999.000,-999,-999,-999,-999,-999.000,35.000,-999,Education,NaN,Good,-3.000,2021-03-12,PAID IN 7,2021-03-09,2021-04-12,1,Friday,12,10am - 12pm,MTN,onoruoyizas.shuaib@gmail.com,Sikiru Shuaib,NaN,NaN,2021-03-12,Renmobile,30.000,NaN,22193860009,25.000,0,0-10
4,90706,0,92002613,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,25,NaN,0.000,Graduate,Ikeja,Lagos,MALE,150000.000,"Arts, entertainment and recreation",19000.000,Single,0,GT Bank,Debit Card (+Cheque),Ikorodu,Renting,1,34,-999,-999,-999.000,-999,-999,-999,-999,-999.000,79.000,-999,"Arts, entertainment and recreation",NaN,Good,-13.000,2021-03-12,PAID IN 7,2021-03-11,2021-05-12,1,Friday,12,4pm - 6pm,AIRTEL,donaldoko9@gmail.com,Donald Okoronkwo,2021-03-11,NaN,2021-03-12,Renmobile,30.000,NaN,22341932161,27.000,0,0-10


In [24]:
del_portf[del_portf['MaxDPDBucket'].isin(['31-60', '61-90', '91-120'])].sample(20).to_csv('Lite Loan IDs for DPD 31-60 & 61-90.csv')


In [25]:
del_portf[del_portf['MaxDPDBucket'].isin(['31-60', '61-90','91-120', '121-Above'])][del_portf['DisbMonth'].isin(['2021-05-01', '2021-06-01', '2021-07-01'])].to_csv('Lite Loan IDs for DPD 31-60 , 61-90 ,91-above.csv')

In [26]:
del_portf[del_portf['MaxDPDBucket'].isin(
    ['31-60', '61-90'])].sample(20).to_csv('Lite Loan IDs for DPD 31-60 & 61-90.csv')


In [27]:
lite_loan_apps.DisbursementDate.max()


Timestamp('2022-02-23 00:00:00')

In [28]:
"""def calc_interval(del_portf):
    if (del_portf.MaxDPD[(del_portf['MaxDPD']<=10)]):
        return '0-10'
    elif (del_portf.MaxDPD[(del_portf['MaxDPD']>=11) & (del_portf['MaxDPD']<=30)]):
        return '11-30'
    elif (del_portf.MaxDPD[(del_portf['MaxDPD']>=31) & (del_portf['MaxDPD']<=60)]):
        return '31-60'
    elif(del_portf.MaxDPD[(del_portf['MaxDPD']>=61) & (del_portf['MaxDPD']<=90)]):
        return '61-90'
    elif(del_portf.MaxDPD[(del_portf['MaxDPD']>=91) & (del_portf['MaxDPD']<=120)]):
        return '91-120'
    elif del_portf.MaxDPD[(del_portf['MaxDPD'])>120]:
        return '121+'
del_portf['DPDBucket'].apply(calc_interval) """


"def calc_interval(del_portf):\n    if (del_portf.MaxDPD[(del_portf['MaxDPD']<=10)]):\n        return '0-10'\n    elif (del_portf.MaxDPD[(del_portf['MaxDPD']>=11) & (del_portf['MaxDPD']<=30)]):\n        return '11-30'\n    elif (del_portf.MaxDPD[(del_portf['MaxDPD']>=31) & (del_portf['MaxDPD']<=60)]):\n        return '31-60'\n    elif(del_portf.MaxDPD[(del_portf['MaxDPD']>=61) & (del_portf['MaxDPD']<=90)]):\n        return '61-90'\n    elif(del_portf.MaxDPD[(del_portf['MaxDPD']>=91) & (del_portf['MaxDPD']<=120)]):\n        return '91-120'\n    elif del_portf.MaxDPD[(del_portf['MaxDPD'])>120]:\n        return '121+'\ndel_portf['DPDBucket'].apply(calc_interval) "

In [29]:
# Lets dimension the delinquency distribution by DisbMonth
DPD_Cat = pd.crosstab(del_portf['DisbMonth'],
                      columns=del_portf['MaxDPDBucket'], dropna=False)
DPD_Cat


MaxDPDBucket,0-10,11-30,121-Above,31-60,61-90,91-120
DisbMonth,,,,,,
2021-02-01,166,12,16,4,2,0
2021-03-01,596,40,9,21,8,0
2021-04-01,582,37,8,18,3,2
2021-05-01,493,29,121,10,3,2
2021-06-01,465,15,118,20,3,0
2021-07-01,337,15,75,6,2,0
2021-08-01,537,25,166,3,5,14
2021-09-01,181,5,59,4,7,3
2021-10-01,474,13,0,14,16,186


In [30]:
# Let's convert DisbMonth from string object to datetime
# Allocate as an index the DisbMonth column and format into a datetime object
DPD_Cat.index = pd.to_datetime(DPD_Cat.index)
# Sort attributes to follow conventional Month-on-month arrangement
DPD_Cat.sort_values(by=['DisbMonth'], inplace=True)
# Re-format the attribute values into a monthname-year format
DPD_Cat.index = DPD_Cat.index.strftime('%b-%Y')
DPD_Cat


MaxDPDBucket,0-10,11-30,121-Above,31-60,61-90,91-120
DisbMonth,,,,,,
Feb-2021,166,12,16,4,2,0
Mar-2021,596,40,9,21,8,0
Apr-2021,582,37,8,18,3,2
May-2021,493,29,121,10,3,2
Jun-2021,465,15,118,20,3,0
Jul-2021,337,15,75,6,2,0
Aug-2021,537,25,166,3,5,14
Sep-2021,181,5,59,4,7,3
Oct-2021,474,13,0,14,16,186


In [31]:
# Re-allocate the 121 and Above bucket for us to have proper order of the delinquency buckets
DPD_Cat['121-High'] = DPD_Cat['121-Above']
# Now, Drop the 121-Above delinquent bucket
DPD_Cat.drop(['121-Above'], axis=1, inplace=True)
# print out output
DPD_Cat


MaxDPDBucket,0-10,11-30,31-60,61-90,91-120,121-High
DisbMonth,,,,,,
Feb-2021,166,12,4,2,0,16
Mar-2021,596,40,21,8,0,9
Apr-2021,582,37,18,3,2,8
May-2021,493,29,10,3,2,121
Jun-2021,465,15,20,3,0,118
Jul-2021,337,15,6,2,0,75
Aug-2021,537,25,3,5,14,166
Sep-2021,181,5,4,7,3,59
Oct-2021,474,13,14,16,186,0


In [32]:
# Let's have the total default distribution by month
DPD_Cat['TotalDefault'] = DPD_Cat['0-10'] + DPD_Cat['11-30'] + \
    DPD_Cat['31-60'] + DPD_Cat['61-90'] + \
    DPD_Cat['91-120'] + DPD_Cat['121-High']


In [33]:
DPD_Cat


MaxDPDBucket,0-10,11-30,31-60,61-90,91-120,121-High,TotalDefault
DisbMonth,,,,,,,
Feb-2021,166,12,4,2,0,16,200
Mar-2021,596,40,21,8,0,9,674
Apr-2021,582,37,18,3,2,8,650
May-2021,493,29,10,3,2,121,658
Jun-2021,465,15,20,3,0,118,621
Jul-2021,337,15,6,2,0,75,435
Aug-2021,537,25,3,5,14,166,750
Sep-2021,181,5,4,7,3,59,259
Oct-2021,474,13,14,16,186,0,703


In [34]:
DPD_Cat.to_csv('Delinquency Report today.csv')


### Write-Offs/Bad Debts
#### Business Position on Write-Offs 
Under Credit risk, there are some categorizations of accounts that spell bad debt for the business. Accounts that generally exceed 90Above delinquency **i.e. 6 months on book and over** are prime candidates for expected write-off by the business. However, not all 90 days past due accounts are written-off eventually. **Portfolio Team** keeps track of these sort of events and sometimes go by the business decision to write-off at 97 days past due. We need to breakdown our write-off portfolio to understand variations in behavior observed over the sampled period.  

In [35]:
del_portf.columns


Index(['Unnamed: 0', 'BureauData', 'LoanId', 'AccountState', 'AccountSubstate',
       'Product', 'CreationDate', 'DisbursementDate', 'DisbMonth',
       'InitialDecision', 'InitialDecisionCode', 'ReasonCode_01', 'Age',
       'DocumentType', 'DTI', 'EducationStatus', 'EmployerLGA',
       'EmployerState', 'Gender', 'Income', 'Industry', 'LoanAmount',
       'MaritalStatus', 'NoChildren', 'RepaymentBank', 'RepaymentMethod',
       'AddresLGA', 'ResidentialStatus', 'Term', 'TimeEmploymentMM',
       'TotalNumberAccountsInGoodStanding', 'TotalNumberOfAccounts',
       'TotalMonthlyInstallments', 'TotalNumberOfAccountsInArrs',
       'TotalNumberOfAccountsMoreThan90', 'HighestReportedMonthsInArrs',
       'TotalNumberOfJudgments', 'TotalOutstandingDebt', 'TotalTimeAtAddress',
       'WorstStatus', 'BusinessSector', 'EmploymentPosition', 'GBFlag',
       'MaxDPD', '(DueDate) FPD', 'FPD', 'Closeddate', 'MaturityDate',
       'EarlySettlementFlg', 'WeekDay', 'DayOfTheMonth', 'timeindayselect

In [36]:
# Let's create a container for our written off accounts
write_offs = del_portf[del_portf['AccountState'] == "CLOSED_WRITTEN_OFF"]
# write_offs


In [37]:
write_offs.shape


(538, 65)

 Out of 3088 accounts disbursed with expected interest income to earned, only 312 accounts went into write-offs/bad-debt

In [38]:
# Lets create a pivot of all disbursed written-off loans
write_off_cat = write_offs.pivot_table(values='AccountState', index='DisbMonth',
                                       columns='MaxDPDBucket', aggfunc=np.count_nonzero, fill_value=0, margins=False, dropna=True)

write_off_cat


MaxDPDBucket,121-Above,91-120
DisbMonth,,
2021-02-01,1,0
2021-03-01,6,0
2021-04-01,8,1
2021-05-01,120,0
2021-06-01,112,0
2021-07-01,72,0
2021-08-01,158,1
2021-09-01,59,0


In [39]:
# convert the string object to datetime
write_off_cat.index = pd.to_datetime(write_off_cat.index)
# add an additional column
write_off_cat['121-High'] = write_off_cat['121-Above']

write_off_cat.drop(['121-Above'], axis=1, inplace=True)

write_off_cat


MaxDPDBucket,91-120,121-High
DisbMonth,,
2021-02-01,0,1
2021-03-01,0,6
2021-04-01,1,8
2021-05-01,0,120
2021-06-01,0,112
2021-07-01,0,72
2021-08-01,1,158
2021-09-01,0,59


In [40]:
write_off_cat['Write_offs'] = write_off_cat['91-120'] + \
    write_off_cat['121-High']

write_off_cat.index = write_off_cat.index.strftime('%b-%Y')

write_off_cat


MaxDPDBucket,91-120,121-High,Write_offs
DisbMonth,,,
Feb-2021,0,1,1
Mar-2021,0,6,6
Apr-2021,1,8,9
May-2021,0,120,120
Jun-2021,0,112,112
Jul-2021,0,72,72
Aug-2021,1,158,159
Sep-2021,0,59,59


In [41]:
write_off_cat.to_csv('write-off pool Lite loans.csv')


In [42]:
# Let's merge the Take_up_df on DPD_Cat to have all disbursed and default volume breakdown per month (Feb through Aug 2021)
Disb_Sample = disbursed_pop.merge(
    DPD_Cat, left_on='DisbMonth', right_index=True)
# Derive an additional column for total disbursements
Disb_Sample['TotalDisbursed'] = Disb_Sample['DisbFreq']
Disb_Sample.drop(['DisbFreq'], axis=1, inplace=True)


In [43]:
Disb_Sample


,DisbMonth,0-10,11-30,31-60,61-90,91-120,121-High,TotalDefault,TotalDisbursed
0,Feb-2021,166,12,4,2,0,16,200,203
1,Mar-2021,596,40,21,8,0,9,674,690
2,Apr-2021,582,37,18,3,2,8,650,665
3,May-2021,493,29,10,3,2,121,658,681
4,Jun-2021,465,15,20,3,0,118,621,639
5,Jul-2021,337,15,6,2,0,75,435,447
6,Aug-2021,537,25,3,5,14,166,750,768
7,Sep-2021,181,5,4,7,3,59,259,265
8,Oct-2021,474,13,14,16,186,0,703,712
9,Nov-2021,452,20,34,200,0,0,706,718


In [44]:
# Here lets get the total default per delinquency buckets as it increases
Disb_Sample['Default 0+'] = Disb_Sample[['0-10', '11-30',
                                         '31-60', '61-90', '91-120', '121-High']].sum(axis=1)
Disb_Sample['Default 11+'] = Disb_Sample[['11-30',
                                          '31-60', '61-90', '91-120', '121-High']].sum(axis=1)
Disb_Sample['Default 31+'] = Disb_Sample[['31-60',
                                          '61-90', '91-120', '121-High']].sum(axis=1)
Disb_Sample['Default 61+'] = Disb_Sample['61-90'] + \
    Disb_Sample['91-120'] + Disb_Sample['121-High']
Disb_Sample['Default 91+'] = Disb_Sample['91-120'] + Disb_Sample['121-High']
Disb_Sample['Default 121+'] = Disb_Sample[['121-High']].sum(axis=1)


In [45]:
Disb_Sample


,DisbMonth,0-10,11-30,31-60,61-90,91-120,121-High,TotalDefault,TotalDisbursed,Default 0+,Default 11+,Default 31+,Default 61+,Default 91+,Default 121+
0,Feb-2021,166,12,4,2,0,16,200,203,200,34,22,18,16,16
1,Mar-2021,596,40,21,8,0,9,674,690,674,78,38,17,9,9
2,Apr-2021,582,37,18,3,2,8,650,665,650,68,31,13,10,8
3,May-2021,493,29,10,3,2,121,658,681,658,165,136,126,123,121
4,Jun-2021,465,15,20,3,0,118,621,639,621,156,141,121,118,118
5,Jul-2021,337,15,6,2,0,75,435,447,435,98,83,77,75,75
6,Aug-2021,537,25,3,5,14,166,750,768,750,213,188,185,180,166
7,Sep-2021,181,5,4,7,3,59,259,265,259,78,73,69,62,59
8,Oct-2021,474,13,14,16,186,0,703,712,703,229,216,202,186,0
9,Nov-2021,452,20,34,200,0,0,706,718,706,254,234,200,0,0


In [46]:
# Firstly, we rename the TotalDefault column as Total Disbursed since we are not excluding ES1 anymore for lite
Disb_Sample = Disb_Sample.rename(columns={'TotalDefault': 'TotalNonES1'})
# Lets obtain our default rates across the different delinquency buckets
Disb_Sample['DR 0+'] = Disb_Sample['Default 0+']/Disb_Sample['TotalNonES1']
Disb_Sample['DR 11+'] = Disb_Sample['Default 11+']/Disb_Sample['TotalNonES1']
Disb_Sample['DR 31+'] = Disb_Sample['Default 31+']/Disb_Sample['TotalNonES1']
Disb_Sample['DR 61+'] = Disb_Sample['Default 61+']/Disb_Sample['TotalNonES1']
Disb_Sample['DR 91+'] = Disb_Sample['Default 91+']/Disb_Sample['TotalNonES1']
Disb_Sample['DR 121+'] = Disb_Sample['Default 121+']/Disb_Sample['TotalNonES1']
# print output
# Disb_Sample=Disb_Sample.rename(columns={'TotalDefault':'TotalDisbursed'})
Disb_Sample.to_csv('Disb_Sample - Lite.csv')


In [47]:
#Disb_Sample = Disb_Sample.rename(columns={'TotalNonES1':'TotalDisbursed'})


In [48]:
#Disb_Sample.to_csv('Updated Disbursed Sample FEB - SEP 2021 today.csv')


In [49]:
# Lets merge the disbursed sample with the written-off loan portfolio
woff_df = Disb_Sample[['DisbMonth', 'TotalDisbursed', '121-High']].merge(
    write_off_cat['Write_offs'], left_on='DisbMonth', right_index=True)

# Let's derive the write-off rates from the total disbursed per vintage
woff_df['write_off_rates'] = woff_df['Write_offs']/woff_df['TotalDisbursed']
woff_df['Diff 121-High & Woff'] = woff_df['121-High'] - woff_df['Write_offs']


In [50]:
Disb_Sample = Disb_Sample.rename(columns={'TotalNonES1': 'TotalDisbursed'})


In [51]:
woff_df


,DisbMonth,TotalDisbursed,121-High,Write_offs,write_off_rates,Diff 121-High & Woff
0,Feb-2021,203,16,1,0.005,15
1,Mar-2021,690,9,6,0.009,3
2,Apr-2021,665,8,9,0.014,-1
3,May-2021,681,121,120,0.176,1
4,Jun-2021,639,118,112,0.175,6
5,Jul-2021,447,75,72,0.161,3
6,Aug-2021,768,166,159,0.207,7
7,Sep-2021,265,59,59,0.223,0


#### Cure Rates Analysis 
**What is Cure Rate in Credit Risk Management ?**

Cure Rates involves disbursed account samples that have either defaulted before or not but ended up repaying in full. **i.e.** Successfully paying off their loans. So when accounts close off successfully they are termed **"cured"**. In the analysis above we have identified disbursed accounts and also their delinquency patterns of dayspastdue buckets from **(0-10, 11-30, 31-60, 61-90, 91-120, 121-High etc.)**. Hence, we will consider this sample for our analysis on lite-loans cure rate performance.


For us to identify specific accounts that were cured. below are the following criterias considered:
* Accounts that have their closed date field populated.
* Accounts that have **"CLOSED"** account states and **NULL/NaNs"** account substates.



In [52]:
# Let's see the distribution of attributes for our Account State feature
del_portf['AccountState'].value_counts()


CLOSED                5157
ACTIVE_IN_ARREARS      680
CLOSED_WRITTEN_OFF     538
ACTIVE                  19
Name: AccountState, dtype: int64

In [53]:
# Let's see the distribution of attributes for our Account Substate field
del_portf['AccountSubstate'].unique()


array([nan, 'LOCKED'], dtype=object)

#### Important Notes to be Considered When doing Cure Rates Analysis
The above unique distribution outputted for account substate shows us there are three possible system generated values:
(1. NaN 
(2. Locked
(3. Withdrawn

(1. **Nan:** an account substate has is Nan means missing values. In the event of closures or full repayments being made, loan accounts will have account state populated as **"CLOSED"** and their corresponding account substate as **"NaN"** or **NULL**.

(2. **Locked:** accounts in the **LOCKED** are in review for some reasons which could possibly be a technical glitch. so we do not get to include them in our analysis. 

(3. **Withdrawn:** even though they do not reflect in our disbursed samples, we still need to have a business understanding of it. accounts that fall into this category and have their account states populated as **CLOSED** are accounts that did not take up a successful loan offer.

In [54]:
cured_df = del_portf[(del_portf['AccountState'] == "CLOSED") & (
    del_portf['AccountSubstate'] != "LOCKED")]


In [55]:
cured_df.info()
# Output=False


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5157 entries, 0 to 6540
Data columns (total 65 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Unnamed: 0                         5157 non-null   int64         
 1   BureauData                         5157 non-null   int64         
 2   LoanId                             5157 non-null   int64         
 3   AccountState                       5157 non-null   object        
 4   AccountSubstate                    0 non-null      object        
 5   Product                            5157 non-null   object        
 6   CreationDate                       5157 non-null   object        
 7   DisbursementDate                   5157 non-null   datetime64[ns]
 8   DisbMonth                          5157 non-null   object        
 9   InitialDecision                    5157 non-null   object        
 10  InitialDecisionCode                5

In [56]:
cured_df['AccountState'].value_counts()


CLOSED    5157
Name: AccountState, dtype: int64

In [57]:
cured_df.head()
Output = False


In [58]:
# Lets create a crosstabulation to present the frequencies of how each delinquency bucket fared as regards cure
cured = pd.crosstab(index=cured_df['DisbMonth'],
                    columns=cured_df['MaxDPDBucket'])
cured


MaxDPDBucket,0-10,11-30,121-Above,31-60,61-90,91-120
DisbMonth,,,,,,
2021-02-01,166,12,15,4,2,0
2021-03-01,596,40,3,21,8,0
2021-04-01,582,37,0,18,3,1
2021-05-01,493,29,1,10,3,2
2021-06-01,465,15,6,20,3,0
2021-07-01,337,15,3,6,2,0
2021-08-01,537,25,8,3,5,1
2021-09-01,181,5,0,4,5,0
2021-10-01,473,13,0,9,7,2


In [59]:
# Convert the string object to datetime
cured.index = pd.to_datetime(cured.index)

# We can now format our DisbMonth column/index to show as MonName-YYYY e.g. Feb-2021
cured.index = cured.index.strftime('%b-%Y')

# Lets print outputs
cured


MaxDPDBucket,0-10,11-30,121-Above,31-60,61-90,91-120
DisbMonth,,,,,,
Feb-2021,166,12,15,4,2,0
Mar-2021,596,40,3,21,8,0
Apr-2021,582,37,0,18,3,1
May-2021,493,29,1,10,3,2
Jun-2021,465,15,6,20,3,0
Jul-2021,337,15,3,6,2,0
Aug-2021,537,25,8,3,5,1
Sep-2021,181,5,0,4,5,0
Oct-2021,473,13,0,9,7,2


In [60]:
cured['Cured 0+'] = cured[['0-10', '11-30', '31-60',
                           '61-90', '91-120', '121-Above']].sum(axis=1)
cured['Cured 11+'] = cured[['11-30', '31-60',
                            '61-90', '91-120', '121-Above']].sum(axis=1)
cured['Cured 31+'] = cured[['31-60', '61-90',
                            '91-120', '121-Above']].sum(axis=1)
cured['Cured 61+'] = cured['61-90'] + cured['91-120'] + cured['121-Above']
cured['Cured 91+'] = cured['91-120'] + cured['121-Above']
cured['Cured 121+'] = cured[['121-Above']].sum(axis=1)


In [61]:
# Outputs from the above shows the column names as default categories instead of cured categories. We need to rename as can be seen below
cured = cured.rename(columns={'0-10': 'Cured 0-10', '11-30': 'Cured 11-30', '31-60': 'Cured 31-60',
                     '61-90': 'Cured 61-90', '91-120': 'Cured 91-120', '121-Above': 'Cured 121-High'})
cured


MaxDPDBucket,Cured 0-10,Cured 11-30,Cured 121-High,Cured 31-60,Cured 61-90,Cured 91-120,Cured 0+,Cured 11+,Cured 31+,Cured 61+,Cured 91+,Cured 121+
DisbMonth,,,,,,,,,,,,
Feb-2021,166,12,15,4,2,0,199,33,21,17,15,15
Mar-2021,596,40,3,21,8,0,668,72,32,11,3,3
Apr-2021,582,37,0,18,3,1,641,59,22,4,1,0
May-2021,493,29,1,10,3,2,538,45,16,6,3,1
Jun-2021,465,15,6,20,3,0,509,44,29,9,6,6
Jul-2021,337,15,3,6,2,0,363,26,11,5,3,3
Aug-2021,537,25,8,3,5,1,579,42,17,14,9,8
Sep-2021,181,5,0,4,5,0,195,14,9,5,0,0
Oct-2021,473,13,0,9,7,2,504,31,18,9,2,0


In [62]:
# Lets preview our full disbursed & delinquency population table of our profitable customers i.e: Non-earlysettlement 1 population
cured


MaxDPDBucket,Cured 0-10,Cured 11-30,Cured 121-High,Cured 31-60,Cured 61-90,Cured 91-120,Cured 0+,Cured 11+,Cured 31+,Cured 61+,Cured 91+,Cured 121+
DisbMonth,,,,,,,,,,,,
Feb-2021,166,12,15,4,2,0,199,33,21,17,15,15
Mar-2021,596,40,3,21,8,0,668,72,32,11,3,3
Apr-2021,582,37,0,18,3,1,641,59,22,4,1,0
May-2021,493,29,1,10,3,2,538,45,16,6,3,1
Jun-2021,465,15,6,20,3,0,509,44,29,9,6,6
Jul-2021,337,15,3,6,2,0,363,26,11,5,3,3
Aug-2021,537,25,8,3,5,1,579,42,17,14,9,8
Sep-2021,181,5,0,4,5,0,195,14,9,5,0,0
Oct-2021,473,13,0,9,7,2,504,31,18,9,2,0


In [63]:
Cure_sample = Disb_Sample[['DisbMonth', '0-10', '11-30',
                           '31-60', '61-90', '91-120',
                           '121-High', 'TotalDisbursed', 'Default 0+',
                           'Default 11+', 'Default 31+', 'Default 61+',
                           'Default 91+', 'Default 121+', 'DR 0+', 'DR 11+',
                           'DR 31+', 'DR 61+', 'DR 91+', 'DR 121+']].merge(cured, left_on='DisbMonth', right_index=True)


In [64]:
Cure_sample.head(5)


,DisbMonth,0-10,11-30,31-60,61-90,91-120,121-High,TotalDisbursed,TotalDisbursed,Default 0+,Default 11+,Default 31+,Default 61+,Default 91+,Default 121+,DR 0+,DR 11+,DR 31+,DR 61+,DR 91+,DR 121+,Cured 0-10,Cured 11-30,Cured 121-High,Cured 31-60,Cured 61-90,Cured 91-120,Cured 0+,Cured 11+,Cured 31+,Cured 61+,Cured 91+,Cured 121+
0,Feb-2021,166,12,4,2,0,16,200,203,200,34,22,18,16,16,1.000,0.170,0.110,0.090,0.080,0.080,166,12,15,4,2,0,199,33,21,17,15,15
1,Mar-2021,596,40,21,8,0,9,674,690,674,78,38,17,9,9,1.000,0.116,0.056,0.025,0.013,0.013,596,40,3,21,8,0,668,72,32,11,3,3
2,Apr-2021,582,37,18,3,2,8,650,665,650,68,31,13,10,8,1.000,0.105,0.048,0.020,0.015,0.012,582,37,0,18,3,1,641,59,22,4,1,0
3,May-2021,493,29,10,3,2,121,658,681,658,165,136,126,123,121,1.000,0.251,0.207,0.191,0.187,0.184,493,29,1,10,3,2,538,45,16,6,3,1
4,Jun-2021,465,15,20,3,0,118,621,639,621,156,141,121,118,118,1.000,0.251,0.227,0.195,0.190,0.190,465,15,6,20,3,0,509,44,29,9,6,6


In [65]:
# Lets add a new column Cured 121 Above which is Cured 121+ category and make it our last column
Cure_sample['Cured 121 Above'] = Cure_sample['Cured 121+']
# Lets drop the initial column Cured 121+ so we do not have duplicates of the same column
Cure_sample.drop(columns=['Cured 121+'], inplace=True, axis=1)
# Finally, we rename the last column as Cured 121+
Cure_sample.rename(columns={'Cured 121 Above': 'Cured 121+'})


,DisbMonth,0-10,11-30,31-60,61-90,91-120,121-High,TotalDisbursed,TotalDisbursed,Default 0+,Default 11+,Default 31+,Default 61+,Default 91+,Default 121+,DR 0+,DR 11+,DR 31+,DR 61+,DR 91+,DR 121+,Cured 0-10,Cured 11-30,Cured 121-High,Cured 31-60,Cured 61-90,Cured 91-120,Cured 0+,Cured 11+,Cured 31+,Cured 61+,Cured 91+,Cured 121+
0,Feb-2021,166,12,4,2,0,16,200,203,200,34,22,18,16,16,1.000,0.170,0.110,0.090,0.080,0.080,166,12,15,4,2,0,199,33,21,17,15,15
1,Mar-2021,596,40,21,8,0,9,674,690,674,78,38,17,9,9,1.000,0.116,0.056,0.025,0.013,0.013,596,40,3,21,8,0,668,72,32,11,3,3
2,Apr-2021,582,37,18,3,2,8,650,665,650,68,31,13,10,8,1.000,0.105,0.048,0.020,0.015,0.012,582,37,0,18,3,1,641,59,22,4,1,0
3,May-2021,493,29,10,3,2,121,658,681,658,165,136,126,123,121,1.000,0.251,0.207,0.191,0.187,0.184,493,29,1,10,3,2,538,45,16,6,3,1
4,Jun-2021,465,15,20,3,0,118,621,639,621,156,141,121,118,118,1.000,0.251,0.227,0.195,0.190,0.190,465,15,6,20,3,0,509,44,29,9,6,6
5,Jul-2021,337,15,6,2,0,75,435,447,435,98,83,77,75,75,1.000,0.225,0.191,0.177,0.172,0.172,337,15,3,6,2,0,363,26,11,5,3,3
6,Aug-2021,537,25,3,5,14,166,750,768,750,213,188,185,180,166,1.000,0.284,0.251,0.247,0.240,0.221,537,25,8,3,5,1,579,42,17,14,9,8
7,Sep-2021,181,5,4,7,3,59,259,265,259,78,73,69,62,59,1.000,0.301,0.282,0.266,0.239,0.228,181,5,0,4,5,0,195,14,9,5,0,0
8,Oct-2021,474,13,14,16,186,0,703,712,703,229,216,202,186,0,1.000,0.326,0.307,0.287,0.265,0.000,473,13,0,9,7,2,504,31,18,9,2,0
9,Nov-2021,452,20,34,200,0,0,706,718,706,254,234,200,0,0,1.000,0.360,0.331,0.283,0.000,0.000,447,16,0,15,3,0,481,34,18,3,0,0


In [66]:
Cure_sample = Cure_sample.rename(columns={'Cured 121 Above': 'Cured 121+'})


In [67]:
# Now, lets evaluate the cure rates per vintages Feb-2021 through Aug-2021
Cure_sample['CR 0+'] = Cure_sample['Cured 0+']/Cure_sample['Default 0+']
Cure_sample['CR 11+'] = Cure_sample['Cured 11+']/Cure_sample['Default 11+']
Cure_sample['CR 31+'] = Cure_sample['Cured 31+']/Cure_sample['Default 31+']
Cure_sample['CR 61+'] = Cure_sample['Cured 61+']/Cure_sample['Default 61+']
Cure_sample['CR 91+'] = Cure_sample['Cured 91+']/Cure_sample['Default 91+']
Cure_sample['CR 121+'] = Cure_sample['Cured 121+']/Cure_sample['Default 121+']

Cure_sample


,DisbMonth,0-10,11-30,31-60,61-90,91-120,121-High,TotalDisbursed,TotalDisbursed,Default 0+,Default 11+,Default 31+,Default 61+,Default 91+,Default 121+,DR 0+,DR 11+,DR 31+,DR 61+,DR 91+,DR 121+,Cured 0-10,Cured 11-30,Cured 121-High,Cured 31-60,Cured 61-90,Cured 91-120,Cured 0+,Cured 11+,Cured 31+,Cured 61+,Cured 91+,Cured 121+,CR 0+,CR 11+,CR 31+,CR 61+,CR 91+,CR 121+
0,Feb-2021,166,12,4,2,0,16,200,203,200,34,22,18,16,16,1.000,0.170,0.110,0.090,0.080,0.080,166,12,15,4,2,0,199,33,21,17,15,15,0.995,0.971,0.955,0.944,0.938,0.938
1,Mar-2021,596,40,21,8,0,9,674,690,674,78,38,17,9,9,1.000,0.116,0.056,0.025,0.013,0.013,596,40,3,21,8,0,668,72,32,11,3,3,0.991,0.923,0.842,0.647,0.333,0.333
2,Apr-2021,582,37,18,3,2,8,650,665,650,68,31,13,10,8,1.000,0.105,0.048,0.020,0.015,0.012,582,37,0,18,3,1,641,59,22,4,1,0,0.986,0.868,0.710,0.308,0.100,0.000
3,May-2021,493,29,10,3,2,121,658,681,658,165,136,126,123,121,1.000,0.251,0.207,0.191,0.187,0.184,493,29,1,10,3,2,538,45,16,6,3,1,0.818,0.273,0.118,0.048,0.024,0.008
4,Jun-2021,465,15,20,3,0,118,621,639,621,156,141,121,118,118,1.000,0.251,0.227,0.195,0.190,0.190,465,15,6,20,3,0,509,44,29,9,6,6,0.820,0.282,0.206,0.074,0.051,0.051
5,Jul-2021,337,15,6,2,0,75,435,447,435,98,83,77,75,75,1.000,0.225,0.191,0.177,0.172,0.172,337,15,3,6,2,0,363,26,11,5,3,3,0.834,0.265,0.133,0.065,0.040,0.040
6,Aug-2021,537,25,3,5,14,166,750,768,750,213,188,185,180,166,1.000,0.284,0.251,0.247,0.240,0.221,537,25,8,3,5,1,579,42,17,14,9,8,0.772,0.197,0.090,0.076,0.050,0.048
7,Sep-2021,181,5,4,7,3,59,259,265,259,78,73,69,62,59,1.000,0.301,0.282,0.266,0.239,0.228,181,5,0,4,5,0,195,14,9,5,0,0,0.753,0.179,0.123,0.072,0.000,0.000
8,Oct-2021,474,13,14,16,186,0,703,712,703,229,216,202,186,0,1.000,0.326,0.307,0.287,0.265,0.000,473,13,0,9,7,2,504,31,18,9,2,0,0.717,0.135,0.083,0.045,0.011,NaN
9,Nov-2021,452,20,34,200,0,0,706,718,706,254,234,200,0,0,1.000,0.360,0.331,0.283,0.000,0.000,447,16,0,15,3,0,481,34,18,3,0,0,0.681,0.134,0.077,0.015,NaN,NaN


In [68]:
lite_loan_apps[lite_loan_apps['AccountState'] ==
               'CLOSED_WRITTEN_OFF'].DisbMonth.value_counts()


2021-08-01    159
2021-05-01    120
2021-06-01    112
2021-07-01     72
2021-09-01     59
2021-04-01      9
2021-03-01      6
2021-02-01      1
Name: DisbMonth, dtype: int64

In [69]:
Cure_sample.to_csv('Cured Sample - Lite Loans.csv')


In [70]:
lite_loan_apps.head()


,Unnamed: 0,BureauData,LoanId,AccountState,AccountSubstate,Product,CreationDate,DisbursementDate,DisbMonth,InitialDecision,InitialDecisionCode,ReasonCode_01,Age,DocumentType,DTI,EducationStatus,EmployerLGA,EmployerState,Gender,Income,Industry,LoanAmount,MaritalStatus,NoChildren,RepaymentBank,RepaymentMethod,AddresLGA,ResidentialStatus,Term,TimeEmploymentMM,TotalNumberAccountsInGoodStanding,TotalNumberOfAccounts,TotalMonthlyInstallments,TotalNumberOfAccountsInArrs,TotalNumberOfAccountsMoreThan90,HighestReportedMonthsInArrs,TotalNumberOfJudgments,TotalOutstandingDebt,TotalTimeAtAddress,WorstStatus,BusinessSector,EmploymentPosition,GBFlag,MaxDPD,(DueDate) FPD,FPD,Closeddate,MaturityDate,EarlySettlementFlg,WeekDay,DayOfTheMonth,timeindayselect,MobileNetwork,Email,ClientFullName,FirstRD,CreditScore,FirstDD,Channels,InterestRate,Acct_Duratn,clientID,Loan duration,Early Settled
0,219111,0,92002528,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,26,NIN,0.000,Graduate,Ikeja,Lagos,MALE,60000.000,Services,11300.000,Single,0,Zenith Bank,Debit Card (+Cheque),Alimosho,Renting,1,56,-999,-999,-999.000,-999,-999,-999,-999,-999.000,275.000,-999,Services,NaN,Good,0.000,2021-03-12,PAID IN 7,2021-03-12,2021-04-12,1,Friday,12,10am - 12pm,AIRTEL,drezyokoro@gmail.com,David Okoro,NaN,NaN,2021-03-12,Renmobile,30.000,NaN,22163766724,28.000,0
1,103666,0,92002589,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,28,Social Security Card,0.000,Graduate,Ikeja,Lagos,MALE,242000.000,Transportation and storage,20000.000,Single,0,Zenith Bank,Debit Card (+Cheque),Alimosho,Owner,1,54,-999,-999,-999.000,-999,-999,-999,-999,-999.000,313.000,-999,Transportation and storage,NaN,Bad,323.000,2021-03-12,PAID IN 7,2021-03-03,2021-05-12,1,Friday,12,2pm - 4pm,MTN,Chukwuedomichael@gmail.com,Michael Chukwuedo,NaN,NaN,2021-03-12,Web App,30.000,NaN,22150679002,19.000,0
2,103656,0,92002560,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,42,Voters Identity Card,0.000,Graduate,Ikorodu,Lagos,MALE,85000.000,Services,10000.000,Married,1,GT Bank,Debit Card (+Cheque),Oredo,Renting,1,180,-999,-999,-999.000,-999,-999,-999,-999,-999.000,27.000,-999,Services,NaN,Good,6.000,2021-03-12,PAID IN 7,2021-03-18,2021-04-12,1,Friday,12,12pm - 2pm,MTN,popoolababatunde@rocketmail.com,Popoola Babatunde,NaN,NaN,2021-03-12,Web App,30.000,NaN,22161568021,34.000,0
3,103636,0,92002533,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,41,Drivers Licence,0.000,Post Graduate,Zaria,Kaduna,MALE,157567.000,Education,20000.000,Married,4,GT Bank,Debit Card (+Cheque),Kaduna,Owner,1,119,-999,-999,-999.000,-999,-999,-999,-999,-999.000,35.000,-999,Education,NaN,Good,-3.000,2021-03-12,PAID IN 7,2021-03-09,2021-04-12,1,Friday,12,10am - 12pm,MTN,onoruoyizas.shuaib@gmail.com,Sikiru Shuaib,NaN,NaN,2021-03-12,Renmobile,30.000,NaN,22193860009,25.000,0
4,90706,0,92002613,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,25,NaN,0.000,Graduate,Ikeja,Lagos,MALE,150000.000,"Arts, entertainment and recreation",19000.000,Single,0,GT Bank,Debit Card (+Cheque),Ikorodu,Renting,1,34,-999,-999,-999.000,-999,-999,-999,-999,-999.000,79.000,-999,"Arts, entertainment and recreation",NaN,Good,-13.000,2021-03-12,PAID IN 7,2021-03-11,2021-05-12,1,Friday,12,4pm - 6pm,AIRTEL,donaldoko9@gmail.com,Donald Okoronkwo,2021-03-11,NaN,2021-03-12,Renmobile,30.000,NaN,22341932161,27.000,0


In [71]:

disb_loans = lite_loan_apps.dropna(subset=['DisbMonth'])


In [72]:
disb_accepted_loans = disb_loans[(disb_loans['InitialDecisionCode'] == "Accept") & (
    disb_loans['Early Settled'] != 1)]


In [73]:
disb_accepted_loans.head()


,Unnamed: 0,BureauData,LoanId,AccountState,AccountSubstate,Product,CreationDate,DisbursementDate,DisbMonth,InitialDecision,InitialDecisionCode,ReasonCode_01,Age,DocumentType,DTI,EducationStatus,EmployerLGA,EmployerState,Gender,Income,Industry,LoanAmount,MaritalStatus,NoChildren,RepaymentBank,RepaymentMethod,AddresLGA,ResidentialStatus,Term,TimeEmploymentMM,TotalNumberAccountsInGoodStanding,TotalNumberOfAccounts,TotalMonthlyInstallments,TotalNumberOfAccountsInArrs,TotalNumberOfAccountsMoreThan90,HighestReportedMonthsInArrs,TotalNumberOfJudgments,TotalOutstandingDebt,TotalTimeAtAddress,WorstStatus,BusinessSector,EmploymentPosition,GBFlag,MaxDPD,(DueDate) FPD,FPD,Closeddate,MaturityDate,EarlySettlementFlg,WeekDay,DayOfTheMonth,timeindayselect,MobileNetwork,Email,ClientFullName,FirstRD,CreditScore,FirstDD,Channels,InterestRate,Acct_Duratn,clientID,Loan duration,Early Settled
0,219111,0,92002528,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,26,NIN,0.000,Graduate,Ikeja,Lagos,MALE,60000.000,Services,11300.000,Single,0,Zenith Bank,Debit Card (+Cheque),Alimosho,Renting,1,56,-999,-999,-999.000,-999,-999,-999,-999,-999.000,275.000,-999,Services,NaN,Good,0.000,2021-03-12,PAID IN 7,2021-03-12,2021-04-12,1,Friday,12,10am - 12pm,AIRTEL,drezyokoro@gmail.com,David Okoro,NaN,NaN,2021-03-12,Renmobile,30.000,NaN,22163766724,28.000,0
1,103666,0,92002589,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,28,Social Security Card,0.000,Graduate,Ikeja,Lagos,MALE,242000.000,Transportation and storage,20000.000,Single,0,Zenith Bank,Debit Card (+Cheque),Alimosho,Owner,1,54,-999,-999,-999.000,-999,-999,-999,-999,-999.000,313.000,-999,Transportation and storage,NaN,Bad,323.000,2021-03-12,PAID IN 7,2021-03-03,2021-05-12,1,Friday,12,2pm - 4pm,MTN,Chukwuedomichael@gmail.com,Michael Chukwuedo,NaN,NaN,2021-03-12,Web App,30.000,NaN,22150679002,19.000,0
2,103656,0,92002560,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,42,Voters Identity Card,0.000,Graduate,Ikorodu,Lagos,MALE,85000.000,Services,10000.000,Married,1,GT Bank,Debit Card (+Cheque),Oredo,Renting,1,180,-999,-999,-999.000,-999,-999,-999,-999,-999.000,27.000,-999,Services,NaN,Good,6.000,2021-03-12,PAID IN 7,2021-03-18,2021-04-12,1,Friday,12,12pm - 2pm,MTN,popoolababatunde@rocketmail.com,Popoola Babatunde,NaN,NaN,2021-03-12,Web App,30.000,NaN,22161568021,34.000,0
3,103636,0,92002533,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,41,Drivers Licence,0.000,Post Graduate,Zaria,Kaduna,MALE,157567.000,Education,20000.000,Married,4,GT Bank,Debit Card (+Cheque),Kaduna,Owner,1,119,-999,-999,-999.000,-999,-999,-999,-999,-999.000,35.000,-999,Education,NaN,Good,-3.000,2021-03-12,PAID IN 7,2021-03-09,2021-04-12,1,Friday,12,10am - 12pm,MTN,onoruoyizas.shuaib@gmail.com,Sikiru Shuaib,NaN,NaN,2021-03-12,Renmobile,30.000,NaN,22193860009,25.000,0
4,90706,0,92002613,CLOSED,NaN,Lite Loan - New,2021-02-12,2021-02-12,2021-02-01,2021-02-12,Accept,NaN,25,NaN,0.000,Graduate,Ikeja,Lagos,MALE,150000.000,"Arts, entertainment and recreation",19000.000,Single,0,GT Bank,Debit Card (+Cheque),Ikorodu,Renting,1,34,-999,-999,-999.000,-999,-999,-999,-999,-999.000,79.000,-999,"Arts, entertainment and recreation",NaN,Good,-13.000,2021-03-12,PAID IN 7,2021-03-11,2021-05-12,1,Friday,12,4pm - 6pm,AIRTEL,donaldoko9@gmail.com,Donald Okoronkwo,2021-03-11,NaN,2021-03-12,Renmobile,30.000,NaN,22341932161,27.000,0


In [74]:
disb_accepted_loans['LoanBand'] = pd.qcut(
    disb_accepted_loans['LoanAmount'], q=4, duplicates='drop')


In [75]:
disb_accepted_loans['LoanBand'].value_counts()


(15300.0, 20000.0]     3188
(3999.999, 10000.0]    1939
(10000.0, 15300.0]     1268
Name: LoanBand, dtype: int64

In [76]:
#disb_accepts = disb_accepted_loans.pivot_table(values='AccountState', index='LoanBand', columns='DisbMonth', aggfunc=np.count_nonzero ,fill_value=0, margins=False, dropna=True)


disb_accepts = disb_accepted_loans.pivot_table(
    values='AccountState', index='LoanBand', columns='DisbMonth', aggfunc=np.count_nonzero, fill_value=0, margins=False, dropna=True)

# Sort attributes to follow conventional Month-on-month arrangement
#disb_accepts.index = pd.to_datetime(disb_accepts.index)
#disb_accepts.sort_values(by=['DisbMonth'], inplace=True)


#disb_accepts['DisbMonth'] = pd.to_datetime(disb_accepts['DisbMonth'])

#disb_accepts['DisbMonth'] = disb_accepts['DisbMonth'].dt.strftime('%b-%Y')


# disb_accepts


In [77]:
# Computing proportions per month across the ticket bands
def func(x): return 100*x.count()/disb_accepted_loans.shape[0]


pd.pivot_table(disb_accepted_loans, index='LoanBand',
               columns='DisbMonth', values='LoanId', aggfunc=func)


DisbMonth,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01
LoanBand,,,,,,,,,,,,,
"(3999.999, 10000.0]",0.829,3.362,3.331,3.456,3.190,2.048,3.471,1.376,3.362,2.658,3.049,0.188,NaN
"(10000.0, 15300.0]",0.485,2.236,2.189,2.174,2.189,1.439,2.346,0.641,2.127,1.955,1.955,0.094,NaN
"(15300.0, 20000.0]",1.814,4.941,4.644,4.660,4.332,3.315,5.911,2.033,5.504,6.427,5.833,0.422,0.016


In [78]:
disb_accepts


DisbMonth,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01
LoanBand,,,,,,,,,,,,,
"(3999.999, 10000.0]",53,215,213,221,204,131,222,88,215,170,195,12,0
"(10000.0, 15300.0]",31,143,140,139,140,92,150,41,136,125,125,6,0
"(15300.0, 20000.0]",116,316,297,298,277,212,378,130,352,411,373,27,1


In [79]:
lite_loan_apps.shape

(6395, 64)

In [84]:
lite_loan_apps= lite_loan_apps[lite_loan_apps['DisbursementDate'].between('2021-05-01', '2021-08-31')]

                            

In [85]:
lite_loan_apps.DisbMonth.value_counts()

2021-08-01    750
2021-05-01    658
2021-06-01    621
2021-07-01    435
Name: DisbMonth, dtype: int64

In [ ]:
lite_loan_apps.drop('GBFlag', axis=1, inplace=True)

In [86]:
lite_loan_apps.to_csv('\\\\10.0.10.20\\NetworkShare\\1. Active Projects\\Lite loans sc\\data\\processed\\2.0 lite loans after behavioral analysis.csv', index=False)

In [87]:
lite_loan_apps.shape

(2464, 64)

In [88]:
lite_loan_apps.clientID.dtype

dtype('int64')